Jupyter notebook for doing the interview task for UMC:

**Analyse the drug side-effect reports (def:_narratives_)**

- look at differences between COVID and non-COVID cases
- what can we say about the text?
- suggestions in assignment:
    - first person? why?
    - redundant narrative? why?
- self investigations
    - text lengths and distributions wrt cases -- classifier. Histogram as feature for a Naive Bayes
        - Statistics
    - What features make it first person?
        - Statistics
    - redundancy detection:
        - duplication detection regexp within narrative
        - duplication of information from other columns and data files?
        - fluff content vs. entropy content
    - vocabulary differences: word frequency distributions TF-IDF 
    - content itself: keywords?
    - timing of narrative wrt symptoms?
    - look at all possibilities and their relevant narratives: FN, FP, TP, TN
    - *Sentiment analysis:* TextBlob, Vader, LLM?
    - *Topic models with LDA*


In [5]:
import pickle
import pandas as pd
from IPython.display import display
import numpy as np

data_filename = "2020VAERSDATA"
vax_filename = "2020VAERSVAX"
symptoms_filename = "2020VAERSSYMPTOMS"

Read in data and display relevant columns 

In [102]:
read_full_data_FLAG = False
# ---
if read_full_data_FLAG:
    with open(data_filename+".pickle", 'rb') as fh:
        df_data, cols_data = pickle.load(fh)
        display(df_data[cols_data].head(5))
    print("\n\n")
    with open(vax_filename+".pickle", 'rb') as fh:
        df_vax, cols_vax = pickle.load(fh)
        display(df_vax[cols_vax].head(5))
    print("\n\n")
    with open(symptoms_filename+".pickle", 'rb') as fh:
        df_symptoms, cols_symptoms = pickle.load(fh)
        display(df_symptoms[cols_symptoms].head(5))
else:
    with open("data_sample.pickle", 'rb') as fh:
        df_data, df_vax, df_symptoms = pickle.load(fh)


Check the total number of patient entries:

In [103]:
# Data Check
print(df_data[cols_data[0]].shape)
print(df_vax[cols_data[0]].shape)
print(df_symptoms[cols_data[0]].shape)

print(df_data[cols_data[0]].nunique())
print(df_vax[cols_data[0]].nunique())
print(df_symptoms[cols_data[0]].nunique())

print(set(df_data["VAERS_ID"]).symmetric_difference(set(df_vax["VAERS_ID"])))
print(set(df_data["VAERS_ID"]).symmetric_difference(set(df_symptoms["VAERS_ID"])))

(287,)
(320,)
(346,)
287
287
287
set()
set()


In [25]:
# temp
ids = [1123, 1345, 2345, 8876]
names = ["Karaboudjan", "covid-19", "co vid", "COVID-19" ]
dict4df = {
    "VAERS_ID": ids,
    "VAERS_VAX":names
}
df_toy = pd.DataFrame(dict4df)
display(df_toy)

,VAERS_ID,VAERS_VAX
0,1123,Karaboudjan
1,1345,covid-19
2,2345,co vid
3,8876,COVID-19


In [70]:
# get all rows where column is related to covid
query_keys = ["covid", "cov"]
df_covid = df_toy[df_toy["VAERS_VAX"].str.replace(" ", "").str.contains('|'.join(query_keys), case=False, regex=True)]
display(df_covid)

,VAERS_ID,VAERS_VAX
1,1345,covid-19
2,2345,co vid
3,8876,COVID-19


In [105]:
df_covid = df_vax[df_vax[cols_vax[1]].str.replace(" ", "").str.contains('|'.join(query_keys), case=False, regex=True)]
display(df_covid)

(46, 8)

In [89]:
idx_start = 46480
idx_end = 46800
df_d = df_data.iloc[idx_start:idx_end]
df_v = df_vax.iloc[idx_start:idx_end]
display(df_d)
df_covid = df_v[df_v[cols_vax[1]].str.replace(" ", "").str.contains('|'.join(query_keys), case=False, regex=True)]
display(df_covid.shape)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
46480,913201,12/29/2020,AZ,58.0,58.0,NaN,F,NaN,"Pt experienced Weakness, dizziness, vomiting, ...",NaN,...,NaN,"HTN, Type 2 DM, Vit D Def.",NaN,NaN,2,12/29/2020,NaN,NaN,NaN,NKDA
46481,913202,12/29/2020,AZ,37.0,37.0,NaN,F,NaN,Pt developed a rash about an 30mins to an hour...,NaN,...,NaN,NaN,NaN,NaN,2,12/29/2020,NaN,Y,NaN,NKDA
46482,913206,12/29/2020,TX,49.0,49.0,NaN,M,NaN,Bell?s Palsey about 4.5 hours after injection.,NaN,...,NaN,"Diabetes, Hypertension, High Cholesterol",NaN,NaN,2,12/29/2020,NaN,NaN,Y,NaN
46483,913207,12/29/2020,OR,20.0,20.0,NaN,F,NaN,First about 15 minutes after I received the va...,NaN,...,NaN,NaN,NaN,NaN,2,12/29/2020,NaN,NaN,NaN,NaN
46484,913208,12/29/2020,UT,19.0,19.0,NaN,F,NaN,Chills and muscle aches two hours after the va...,NaN,...,NaN,Asthma and environmental (seasonal) allergies,NaN,NaN,2,12/29/2020,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46795,913493,12/30/2020,TN,39.0,39.0,NaN,F,NaN,Started having throat swelling approximately 5...,NaN,...,NaN,Mass Cell Activation Syndrome - was diagnosed ...,NaN,NaN,2,12/30/2020,NaN,NaN,NaN,"Any foods high in histamine - wine, tomatoes, ..."
46796,913494,12/30/2020,TX,40.0,40.0,NaN,M,NaN,Patient started with sore arm the day of the v...,NaN,...,none,Hypertension and heart disease,NaN,NaN,2,12/30/2020,NaN,NaN,NaN,none
46797,913495,12/30/2020,CT,20.0,20.0,NaN,F,NaN,"Nausea, vomiting, vertigo, syncope, chills, he...",NaN,...,NaN,NaN,NaN,NaN,2,12/30/2020,NaN,NaN,NaN,Penicillin allergy
46798,913496,12/30/2020,NC,57.0,57.0,NaN,F,NaN,"dizziness, pounding headache for most of night...",NaN,...,none,none,NaN,NaN,2,12/30/2020,NaN,NaN,NaN,none


(46, 8)